In [175]:
import numpy as np; import pandas as pd; import pyodbc; import datetime; import sklearn
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_distances
from scipy.spatial import distance
%pylab inline 
import sys
import warnings
pd.options.display.max_columns = 999
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
from IPython.core.display import display,HTML
display(HTML("<style>.container{width:95% !important;}</style>"))

conn = pyodbc.connect(r'DRIVER={SQL Server Native Client 11.0};'r'SERVER=WIN-5G5AUOCEJPK;'r'DATABASE=Gold585;'r'Trusted_Connection=yes;')

Populating the interactive namespace from numpy and matplotlib


In [89]:
#Используемые характеристики номенклатуры:
# GroupID,CollectionID, WeightNet, WearType,MaterialStatusID,PriceSegment,Metal, MetalColor,StylisticGroup,Insert
# Среднюю цену, Суммарную выручку за последние 3 месяца, Количество человек, купивших данную вещь
#вместо группировки по MaterialID делаем по 
sql_query ="Select \
    IIF(model_name is NULL, MaterialID, model_name) ID, \
    Name, \
    GroupID, \
    CollectionID, \
    WeightNet, \
    WearType, \
    MaterialStatusID, \
    PriceSegment, \
    Metal, \
    MetalColor, \
    StylisticGroup, \
    Ins, \
    SUM(SummaAfterDiscount) SUM_Price, \
    AVG(SummaAfterDiscount) AVG_Price, \
    COUNT(PartnerID) Qnt \
    from Purchase \
    Where DATEDIFF(MONTH,PurchaseDate,GETDATE()) <=3 and NAME NOT LIKE('%б/у') and WearType !='' and WearType Not in ('ФУТЛЯР','ЧИСТЯЩЕЕ СРЕДСТВО') and Price > 0 \
    Group by IIF(model_name is NULL, MaterialID, model_name), Name, GroupID,CollectionID, WearType,MaterialStatusID,PriceSegment,Metal, MetalColor,StylisticGroup,Ins,WeightNet \
    Order by COUNT(PartnerID) DESC"

df = pd.read_sql(sql_query,conn)
df.head()

,ID,Name,GroupID,CollectionID,WeightNet,WearType,MaterialStatusID,PriceSegment,Metal,MetalColor,StylisticGroup,Ins,SUM_Price,AVG_Price,Qnt
0,402-002-118,Серьги с алмазной гранью,216,1598,2.031,СЕРЬГИ,3,ДОРОГОЙ,ЗОЛОТО,КРАСНЫЙ,КЛАССИКА,,5591052.60,5546.679167,1008
1,402-002-934,"Серьги, БК",216,1598,1.200,СЕРЬГИ,3,СРЕДНИЙ,ЗОЛОТО,КРАСНЫЙ,КЛАССИКА,,3214706.00,3524.896930,912
2,702-005-828,"Серьги, Фиан., родир.",257,1202,2.513,СЕРЬГИ,3,ДОРОГОЙ,СЕРЕБРО,БЕЛЫЙ,КЛАССИКА,ФИАНИТ,582404.80,718.131689,811
3,202-000-125,Серьги с топазом+фианит,241,450,1.745,СЕРЬГИ,3,ДЕШЕВЫЙ,ЗОЛОТО,КРАСНЫЙ,КЛАССИКА,ТОПАЗ,3686735.84,5085.152883,725
4,702-004-777,"Серьги с фиан., родир.",257,729,2.074,СЕРЬГИ,3,ДЕШЕВЫЙ,СЕРЕБРО,БЕЛЫЙ,КЛАССИКА,ФИАНИТ,334294.35,475.525391,703


In [136]:
#Используемые характеристики номенклатуры:
# GroupID,CollectionID, WeightNet, WearType,MaterialStatusID,PriceSegment,Metal, MetalColor,StylisticGroup,Insert
# Среднюю цену, Суммарную выручку за последние 3 месяца, Количество человек, купивших данную вещь
#вместо группировки по MaterialID делаем по 
sql_query ="Select \
    IIF(model_name = '', CAST(MaterialID AS char),model_name) ID, \
    MAX(Name) Name, \
    MAX(GroupID) GroupID, \
    MAX(CollectionID) CollectionID, \
    AVG(WeightNet) WeightNet, \
    WearType, \
    MAX(MaterialStatusID) MaterialStatusID, \
    PriceSegment, \
    Metal, \
    MetalColor, \
    StylisticGroup, \
    Ins, \
    SUM(SummaAfterDiscount) SUM_Price, \
    AVG(SummaAfterDiscount) AVG_Price, \
    COUNT(PartnerID) Qnt \
    from Purchase \
    Where DATEDIFF(MONTH,PurchaseDate,GETDATE()) <=3 and NAME NOT LIKE('%б/у') and WearType !='' and WearType Not in ('ФУТЛЯР','ЧИСТЯЩЕЕ СРЕДСТВО') and Price > 0 \
    Group by IIF(model_name = '', CAST(MaterialID AS char),model_name), WearType,PriceSegment,Metal, MetalColor,StylisticGroup,Ins \
    Order by  COUNT(PartnerID) DESC"

df = pd.read_sql(sql_query,conn)
df.head()

,ID,Name,GroupID,CollectionID,WeightNet,WearType,MaterialStatusID,PriceSegment,Metal,MetalColor,StylisticGroup,Ins,SUM_Price,AVG_Price,Qnt
0,601-003-456,"Кольцо обруч.гладкое, 22 (ш5)",1869,710,1.723367,КОЛЬЦО,3,СРЕДНИЙ,ЗОЛОТО,КРАСНЫЙ,КЛАССИКА,,15588047.44,4743.775849,3286
1,601-003-470,"Кольцо обруч. гладкое 21,5, БК (ш3,5)",1869,1401,1.756025,КОЛЬЦО,3,СРЕДНИЙ,ЗОЛОТО,КРАСНЫЙ,КЛАССИКА,,11560657.73,5124.405022,2256
2,504-003-155,"ЦЕПЬ СИНГАПУР 60; ПОЛН; 0,20; 585 КР",247,0,1.201453,ЦЕПЬ,18,ДЕШЕВЫЙ,ЗОЛОТО,КРАСНЫЙ,БАЗА,,6997574.29,3821.722714,1831
3,601-003-462,"Обруч.кольцо, 23 (ш3)",1869,1367,1.228182,КОЛЬЦО,9,ДЕШЕВЫЙ,ЗОЛОТО,КРАСНЫЙ,КЛАССИКА,,5088256.77,3030.528154,1679
4,804-000-351,"Цепь каучуковая,родир.,55",269,0,0.488728,ЦЕПЬ,3,ДЕШЕВЫЙ,СЕРЕБРО,БЕЛЫЙ,БАЗА,,559108.95,352.972822,1584


In [137]:
df.shape

(31731, 15)

In [155]:
train = df.drop('SUM_Price',axis=1).copy()

In [156]:
# Много значений у GroupID - 134, CollectionID-616

In [157]:
train.head()

,ID,Name,GroupID,CollectionID,WeightNet,WearType,MaterialStatusID,PriceSegment,Metal,MetalColor,StylisticGroup,Ins,AVG_Price,Qnt
0,601-003-456,"Кольцо обруч.гладкое, 22 (ш5)",1869,710,1.723367,КОЛЬЦО,3,СРЕДНИЙ,ЗОЛОТО,КРАСНЫЙ,КЛАССИКА,,4743.775849,3286
1,601-003-470,"Кольцо обруч. гладкое 21,5, БК (ш3,5)",1869,1401,1.756025,КОЛЬЦО,3,СРЕДНИЙ,ЗОЛОТО,КРАСНЫЙ,КЛАССИКА,,5124.405022,2256
2,504-003-155,"ЦЕПЬ СИНГАПУР 60; ПОЛН; 0,20; 585 КР",247,0,1.201453,ЦЕПЬ,18,ДЕШЕВЫЙ,ЗОЛОТО,КРАСНЫЙ,БАЗА,,3821.722714,1831
3,601-003-462,"Обруч.кольцо, 23 (ш3)",1869,1367,1.228182,КОЛЬЦО,9,ДЕШЕВЫЙ,ЗОЛОТО,КРАСНЫЙ,КЛАССИКА,,3030.528154,1679
4,804-000-351,"Цепь каучуковая,родир.,55",269,0,0.488728,ЦЕПЬ,3,ДЕШЕВЫЙ,СЕРЕБРО,БЕЛЫЙ,БАЗА,,352.972822,1584


In [158]:
train.head()

,ID,Name,GroupID,CollectionID,WeightNet,WearType,MaterialStatusID,PriceSegment,Metal,MetalColor,StylisticGroup,Ins,AVG_Price,Qnt
0,601-003-456,"Кольцо обруч.гладкое, 22 (ш5)",1869,710,1.723367,КОЛЬЦО,3,СРЕДНИЙ,ЗОЛОТО,КРАСНЫЙ,КЛАССИКА,,4743.775849,3286
1,601-003-470,"Кольцо обруч. гладкое 21,5, БК (ш3,5)",1869,1401,1.756025,КОЛЬЦО,3,СРЕДНИЙ,ЗОЛОТО,КРАСНЫЙ,КЛАССИКА,,5124.405022,2256
2,504-003-155,"ЦЕПЬ СИНГАПУР 60; ПОЛН; 0,20; 585 КР",247,0,1.201453,ЦЕПЬ,18,ДЕШЕВЫЙ,ЗОЛОТО,КРАСНЫЙ,БАЗА,,3821.722714,1831
3,601-003-462,"Обруч.кольцо, 23 (ш3)",1869,1367,1.228182,КОЛЬЦО,9,ДЕШЕВЫЙ,ЗОЛОТО,КРАСНЫЙ,КЛАССИКА,,3030.528154,1679
4,804-000-351,"Цепь каучуковая,родир.,55",269,0,0.488728,ЦЕПЬ,3,ДЕШЕВЫЙ,СЕРЕБРО,БЕЛЫЙ,БАЗА,,352.972822,1584


In [159]:
(train['MaterialStatusID'].value_counts())

10    8816
3     8315
11    4538
6     2545
18    2197
9     1346
2     1301
12    1018
7      940
8      386
5      217
19      66
1       46
Name: MaterialStatusID, dtype: int64

In [160]:
#'GroupID','CollectionID',
train2  = pd.get_dummies(train,columns=['GroupID','CollectionID','WearType','MaterialStatusID','PriceSegment','Metal','MetalColor','StylisticGroup','Ins'],drop_first=True)
train2['AVG_Price'] = train2['AVG_Price']/1000
train2['Qnt'] = train2['Qnt']/1000
#train2.drop(['GroupID','CollectionID'],axis=1,inplace=True)

In [161]:
train2.head()

,ID,Name,WeightNet,AVG_Price,Qnt,GroupID_215,GroupID_216,GroupID_217,GroupID_218,GroupID_219,GroupID_221,GroupID_222,GroupID_223,GroupID_224,GroupID_225,GroupID_226,GroupID_231,GroupID_232,GroupID_234,GroupID_235,GroupID_236,GroupID_237,GroupID_238,GroupID_239,GroupID_240,GroupID_241,GroupID_242,GroupID_243,GroupID_244,GroupID_245,GroupID_246,GroupID_247,GroupID_248,GroupID_249,GroupID_250,GroupID_252,GroupID_253,GroupID_254,GroupID_255,GroupID_256,GroupID_257,GroupID_258,GroupID_259,GroupID_260,GroupID_261,GroupID_262,GroupID_263,GroupID_264,GroupID_265,GroupID_266,GroupID_267,GroupID_268,GroupID_269,GroupID_272,GroupID_273,GroupID_274,GroupID_284,GroupID_285,GroupID_1690,GroupID_1709,GroupID_1730,GroupID_1731,GroupID_1732,GroupID_1733,GroupID_1734,GroupID_1735,GroupID_1736,GroupID_1738,GroupID_1739,GroupID_1740,GroupID_1741,GroupID_1742,GroupID_1743,GroupID_1744,GroupID_1745,GroupID_1746,GroupID_1747,GroupID_1749,GroupID_1750,GroupID_1751,GroupID_1752,GroupID_1753,GroupID_1778,GroupID_1779,GroupID_1780,GroupID_1781,GroupID_1782,GroupID_1783,GroupID_1785,GroupID_1786,GroupID_1787,GroupID_1788,GroupID_1789,GroupID_1790,GroupID_1791,GroupID_1792,GroupID_1793,GroupID_1794,GroupID_1795,GroupID_1796,GroupID_1797,GroupID_1798,GroupID_1799,GroupID_1800,GroupID_1801,GroupID_1802,GroupID_1803,GroupID_1804,GroupID_1806,GroupID_1807,GroupID_1809,GroupID_1810,GroupID_1812,GroupID_1829,GroupID_1830,GroupID_1846,GroupID_1869,GroupID_1870,GroupID_1871,GroupID_1873,GroupID_1874,GroupID_1875,GroupID_1876,GroupID_1877,GroupID_1878,GroupID_1879,GroupID_1880,GroupID_1881,GroupID_1882,GroupID_1883,GroupID_1886,GroupID_1887,GroupID_1888,GroupID_1889,GroupID_1890,GroupID_1891,GroupID_1892,GroupID_1894,GroupID_1895,GroupID_1896,GroupID_1897,GroupID_1899,GroupID_1900,GroupID_1902,GroupID_1903,GroupID_1904,GroupID_1905,GroupID_2166,GroupID_2167,GroupID_2177,GroupID_2181,GroupID_2182,GroupID_2185,GroupID_2186,GroupID_2187,GroupID_2188,GroupID_2190,GroupID_2191,GroupID_2196,GroupID_2197,GroupID_2198,GroupID_2201,GroupID_2248,GroupID_2381,GroupID_2382,GroupID_2383,GroupID_2384,GroupID_2385,GroupID_2386,GroupID_2387,GroupID_2388,GroupID_2389,GroupID_2390,GroupID_2391,GroupID_2392,GroupID_2393,GroupID_2394,GroupID_2395,GroupID_2396,GroupID_2397,GroupID_2398,GroupID_2407,CollectionID_1,CollectionID_2,CollectionID_3,CollectionID_4,CollectionID_5,CollectionID_6,CollectionID_7,CollectionID_8,CollectionID_9,CollectionID_10,CollectionID_11,CollectionID_12,CollectionID_13,CollectionID_14,CollectionID_15,CollectionID_16,CollectionID_17,CollectionID_18,CollectionID_19,CollectionID_21,CollectionID_22,CollectionID_23,CollectionID_24,CollectionID_25,CollectionID_26,CollectionID_27,CollectionID_28,CollectionID_29,CollectionID_30,CollectionID_31,CollectionID_33,CollectionID_34,CollectionID_35,CollectionID_36,CollectionID_38,CollectionID_39,CollectionID_40,CollectionID_41,CollectionID_42,CollectionID_43,CollectionID_44,CollectionID_45,CollectionID_46,CollectionID_48,CollectionID_49,CollectionID_50,CollectionID_51,CollectionID_52,CollectionID_53,CollectionID_54,CollectionID_55,CollectionID_56,CollectionID_57,CollectionID_58,CollectionID_59,CollectionID_61,CollectionID_62,CollectionID_63,CollectionID_64,CollectionID_66,CollectionID_67,CollectionID_68,CollectionID_69,CollectionID_70,CollectionID_71,CollectionID_72,CollectionID_73,CollectionID_74,CollectionID_75,CollectionID_76,CollectionID_77,CollectionID_78,CollectionID_79,CollectionID_80,CollectionID_81,CollectionID_82,CollectionID_84,CollectionID_86,CollectionID_87,CollectionID_88,CollectionID_89,CollectionID_90,CollectionID_91,CollectionID_92,CollectionID_93,CollectionID_94,CollectionID_95,CollectionID_96,CollectionID_97,CollectionID_98,CollectionID_99,CollectionID_100,CollectionID_101,CollectionID_102,CollectionID_103,CollectionID_104,CollectionID_107,CollectionID_108,CollectionID_109,CollectionID_110,CollectionID_111,CollectionID_112,CollectionID_113,CollectionID_114,CollectionID_115,CollectionID_116,CollectionID_117,

In [162]:
#train2.corrwith()

In [163]:
%%time
X = train2.drop(['ID','Name'],axis=1)
nbrs = NearestNeighbors(n_neighbors=21,metric='cosine') # algorithm='ball_tree',
nbrs.fit(X.values)
distances, indices = nbrs.kneighbors(X)

Wall time: 42.5 s


In [164]:
indices

array([[    0,     1,     6, ...,    86,   606,  1418],
       [    1,     6,     5, ...,  1418,  1553,  1199],
       [    2,  7930, 30300, ...,    83,   885,  2845],
       ...,
       [31728, 20121, 30571, ..., 25136, 24489, 16608],
       [31729, 30243, 29371, ..., 29086, 23807, 30594],
       [31730, 12513, 11505, ..., 18517, 13045, 19061]], dtype=int64)

In [165]:
indices

array([[    0,     1,     6, ...,    86,   606,  1418],
       [    1,     6,     5, ...,  1418,  1553,  1199],
       [    2,  7930, 30300, ...,    83,   885,  2845],
       ...,
       [31728, 20121, 30571, ..., 25136, 24489, 16608],
       [31729, 30243, 29371, ..., 29086, 23807, 30594],
       [31730, 12513, 11505, ..., 18517, 13045, 19061]], dtype=int64)

In [166]:
#5000  - 21.4 s
#10000 - 2min 16s
#20000 - 11min 9s
#30000 - 30min 25s

In [167]:
indices = indices.astype(str)

In [168]:
indices

array([['0', '1', '6', ..., '86', '606', '1418'],
       ['1', '6', '5', ..., '1418', '1553', '1199'],
       ['2', '7930', '30300', ..., '83', '885', '2845'],
       ...,
       ['31728', '20121', '30571', ..., '25136', '24489', '16608'],
       ['31729', '30243', '29371', ..., '29086', '23807', '30594'],
       ['31730', '12513', '11505', ..., '18517', '13045', '19061']],
      dtype='<U21')

In [169]:
%%time
for i in range(indices.shape[0]):
    for j in range(indices.shape[1]):
        indices[i,j] = train2['Name'].loc[indices[i,j].astype(int)]

Wall time: 32.3 s


In [170]:
recomend = pd.DataFrame(indices,columns=['Material','Rec_1','Rec_2','Rec_3','Rec_4','Rec_5','Rec_6','Rec_7','Rec_8','Rec_9','Rec_10'\
                                        ,'Rec_11','Rec_12','Rec_13','Rec_14','Rec_15','Rec_16','Rec_17','Rec_18','Rec_19','Rec_20'])

In [171]:
recomend.head(20) #[50:70]

,Material,Rec_1,Rec_2,Rec_3,Rec_4,Rec_5,Rec_6,Rec_7,Rec_8,Rec_9,Rec_10,Rec_11,Rec_12,Rec_13,Rec_14,Rec_15,Rec_16,Rec_17,Rec_18,Rec_19,Rec_20
0,"Кольцо обруч.гладкое,",Кольцо обруч. гладкое,"Обруч.кольцо, 21,5 (ш",Кольцо обр. гл. 20.5,"Обруч.кольцо, 22 (ш2,","Обручальное кольцо, г","Обручальное кольцо, г",Кольцо обр. гл. 22.0,Кольцо обр. гл. 23.0,Кольцо обр. гл. 21.5,"Обруч.кольцо, 23 (ш3)","Обруч.кольцо, 17,5 (ш","Обруч.кольцо, 23 (ш4)",Кольцо обручальное гл,Кольцо обручальное гл,"Обруч.кольцо, 21,5 (ш",Кольцо обручальное 22,Кольцо обруч. гладкое,"Обручальное кольцо, г",Кольцо обручальное с,Кольцо обручальное 23
1,Кольцо обруч. гладкое,"Обруч.кольцо, 21,5 (ш",Кольцо обр. гл. 20.5,"Кольцо обруч.гладкое,","Обручальное кольцо, г",Кольцо обруч. гладкое,"Обручальное кольцо, г","Обруч.кольцо, 22 (ш2,","Обруч.кольцо, 17,5 (ш",Кольцо обр. гл. 23.0,Кольцо обр. гл. 22.0,Кольцо обручальное гл,"Обруч.кольцо, 21,5 (ш",Кольцо обручальное 22,"Обруч.кольцо, 23 (ш4)",Кольцо обручальное гл,Кольцо обр. гл. 21.5,Кольцо обручальное с,Кольцо обручальное 23,"Кольцо обручальное, 1",Кольцо обруч. гладкое
2,ЦЕПЬ СИНГАПУР 60; ПОЛ,Сингапур ЦЕПЬ 55.0,"Якорь с подвесами 45,","Ромб одинарный ЦЕПЬ,",ЦЕПЬ СИНГАПУР 45; ПУС,ЦЕПЬ ПАНЦИРЬ 50; ПОЛН,ЦЕПЬ СИНГАПУР 50; ПУС,ЦЕПЬ СИНГАПУР 50; ПОЛ,ЦЕПЬ РОМБ 40; ПОЛН; 0,ЦЕПЬ РОМБ 45; ПУСТ; 0,"РОМБ 45,0; ПОЛН; d 0,",ЦЕПЬ РОМБ 50; ПУСТ; 0,ЦЕПЬ СИНГАПУР 55; ПУС,"ЦЕПЬ ЛАВ 45; ПУСТ; 0,",ЦЕПЬ РОМБ 40; ПУСТ; 0,ЦЕПЬ ПАНЦИРЬ 50; ПОЛН,"ЦЕПЬ ЛАВ 50; ПУСТ; 0,",ЦЕПЬ СИНГАПУР 40; ПУС,ЦЕПЬ РОМБ 55; ПОЛН; 0,ЦЕПЬ НОННА 50; ПУСТ;,ЦЕПЬ ЯКОРЬ 40; ПОЛН;
3,"Обруч.кольцо, 23 (ш3)",Кольцо гл. 23.0 (ш2),Кольцо обр. гл. 16.0,Кольцо обр. с алм. гр,Кольцо обруч. гладкое,"Кольцо обруч.гладкое,","Обруч.кольцо, 21,5 (ш","Кольцо гл. 22.5 (ш2,5",Кольцо обручал. гладк,Кольцо обр. гл. 20.5,"Обруч.кольцо, 22 (ш2,","Обручальное кольцо, г","Обручальное кольцо, г","Обручальное кольцо, г","Кольцо БК, 16",Кольцо обручальное 23,Кольцо обр. с алм. гр,Кольцо обр. гл. 22.0,Кольцо обр. гл. 23.0,"Обручальное кольцо, г","Обруч.кольцо, 18,5 (ш"
4,"Цепь каучуковая,родир","Цепь Панцирь с а/о, р","Цепь Панцирь, родир.,","Цепь Сингапур, родир.","Цепь Панцирь, родир.,","Цепь Панцирь, алм.огр","Цепь Панцирь,алм.огр.",Шнурок текстильный 2м,"Цепь Панцирь, родир.,","Цепь Корда, родир., 4","Цепь Сингапур, алм.гр","Цепь Лав,алм.огр.,род","Цепь Ромб двойн.,алм.","Цепь Якорь, родир., а","Цепь Сингапур,алм.огр","Цепь Якорь, родир., 4","Цепь каучуковая,родир","Цепь Панцирная, алм.г","Цепь Якорь, родир., 4","Цепь Якорь, родир., а","Цепь Сингапур с а/о,"
5,Кольцо обр. гл. 20.5,"Обруч.кольцо, 17,5 (ш","Обруч.кольцо, 23 (ш4)","Обручальное кольцо, г","Обруч.кольцо, 21,5 (ш",Кольцо обручальное гл,"Обруч.кольцо, 21,5 (ш",Кольцо обр. гл. 22.0,Кольцо обручальное 22,Кольцо обруч. гладкое,"Кольцо обручальное, 1",Кольцо обруч. гладкое,"Обручальное кольцо, г",Кольцо обручальное 23,Кольцо обруч. гл. 21.,Кольцо обручальное гл,"Кольцо обручальное, 1",Кольцо обручальное 17,Кольцо обручальное с,Кольцо обручальное 23,Кольцо обруч. гладкое
6,"Обруч.кольцо, 21,5 (ш","Обруч.кольцо, 21,5 (ш","Кольцо обр. гл. 20.5,",Кольцо обр. гл. 20.5,Кольцо обруч. гладкое,"Обручальное кольцо, г",Кольцо обруч. гладкое,"Обручальное кольцо, г","Обруч.кольцо, 22 (ш2,","Обруч.кольцо, 17,5 (ш",Кольцо обручальное гл,Кольцо обруч. гладкое,Кольцо обручальное 22,Кольцо обр. гл. 23.0,Кольцо обр. гл. 22.0,Кольцо обручальное с,"Кольцо обручальное, 1",Кольцо обручальное 23,Кольцо обручальное 23,"Кольцо обручальное, 1",Кольцо гл. 23.0
7,"Цепь Бисмарк, чернени","Цепь Бисмарк, чернени","Цепь Бисмарк, черн.,","Цепь Ромб двойной, ал","Цепь Якорь, черн., а/",Цепь Ромб двон. с алм,"Цепь Якорь, черн., а/","Цепь Якорь, черн., а/","Цепь Бисмарк с а/о, ч",Цепь Панцирь с алм.ог,"Цепь Якорная, алм.гра","Цепь Бисмарк с а/о, ч","Цепь Якорь, черн., а/",Цепь Панцирь с алм.ог,"Цепь Бисмарк, чернени",Цепь Панцирь с алм.ог,"Цепь Бисмарк, чернени","Цепь Ромб двойной, че","Цепь Бисмарк с а/о, ч",Цепь Панцирь с алм.ог,"Цепь Панцирь, родир.,"
8,"Обручальное 

In [172]:
recomend[450:470]

,Material,Rec_1,Rec_2,Rec_3,Rec_4,Rec_5,Rec_6,Rec_7,Rec_8,Rec_9,Rec_10,Rec_11,Rec_12,Rec_13,Rec_14,Rec_15,Rec_16,Rec_17,Rec_18,Rec_19,Rec_20
450,Подвес-полумесяц,Подвес Полумесяц,Подвес культовый,Подвес культовый,Подвес культовый,Подвес культовый,Подвес культовый,Подвес культовый,Крест алм.гр.,Крест алм.гр.,Крест (лазер),Подвес Крест БК,Крест,Крест с род.,Подвес БК,Крест,Крест,Подвес Икона Казанска,Крест,Крест,Подвес Крест БК
451,БРАСЛЕТ КОРДА 17; ПУС,БРАСЛЕТ БИСМАРК 19; П,БРАСЛЕТ БИСМАРК 16; П,СИНГАПУР 18; ПУСТ; d,БРАСЛЕТ РОМБ 16; ПУСТ,БРАСЛЕТ РОМБ 19; ПУСТ,БРАСЛЕТ на нити б/р;,БРАСЛЕТ СИНГАПУР 20;,БРАСЛЕТ ФАНТАЗИЙНЫЙ 1,БРАСЛЕТ ФАНТАЗИЙНЫЙ 1,"РОМБ 20,0; ПУСТ; d 0,",БРАСЛЕТ БИСМАРК 19; П,"Ромб двойной БРАСЛЕТ,",БРАСЛЕТ РОМБ 18; ПУСТ,БРАСЛЕТ ФАНТАЗИЙНЫЙ 1,БРАСЛЕТ ФАНТАЗИЙНЫЙ 1,БРАСЛЕТ ЛАВ 16; ПОЛН;,БРАСЛЕТ ПАНЦИРЬ 17; П,"РОМБ 19,0; ПУСТ; d 0,",Браслет Лав 20.0,БРАСЛЕТ ФАНТАЗИЙНЫЙ 1
452,"Подвес, Буква А","Подвес, Буква В","Подвес, Буква Е","Подвес, Буква И","Подвес, Буква С",Подвеска Буква А,Подвеска Буква А,Подвеска Буква А,Подвеска Буква Е,Подвеска Буква Н,Подвеска Буква В,"Подвес, Буква О","Подвес, Буква К",Подвес,"Крест, родир.","Крест 10*15, родир.",Крест,Знак Зодиака Скорпион,Крест,Крест,Крест
453,"Кольцо с фианитами, 1","Кольцо ИФ, 18","Кольцо ИФ, 16,5","Кольцо с фианитами, 1","Кольцо с фианитами, 1","Кольцо ИФ, 17,5","Кольцо с фианитами, 1","Кольцо ИФ, 17","Кольцо ИФ, 18","Кольцо ИФ, 16,5","Кольцо ИФ, 17,5","Кольцо ИФ, 18,5","Кольцо ИФ, 17","Кольцо ИФ, 17,5","Кольцо с фианитами, 1","Кольцо ИФ, 16","Кольцо ИФ, 18","Кольцо ИФ, 19","Кольцо ИФ, 16","Кольцо ИФ, 17,5","Кольцо с фианитами, 1"
454,"Кольцо с брилл, 17,5","Кольцо с брилл, 16,0",Кольцо с бриллиантом,"Кольцо с брилл. 17,5","Кольцо с бриллиантом,",Кольцо с бриллиантом,"Кольцо с брилл,16,5","Кольцо с брилл,17,0",Кольцо с бриллиантами,Кольцо с бриллиантами,Кольцо с брилл 18.0,Кольцо с брилл 16.0 1,Кольцо с бриллиантами,Кольцо с бриллиантами,Кольцо с бриллиантами,Кольцо с бриллиантом,Кольцо с бриллиантом,Кольцо с бриллиантами,Кольцо с бриллиантом,кольцо с брилл 17.5 4,Кольцо с брилл 17.0
455,"Серьги, иск. изумр. и","Серьги, иск. изумр. и","Серьги, иск. изумр. и","Серьги, иск. изумр. и","Серьги, иск. изумр. и","Серьги, иск. изумр. и","Серьги, иск. сапф. и","Серьги, иск. сапф. и","Серьги, иск. сапф. и","Серьги, иск. рубин. и","Серьги, иск. сапф. и","Серьги, иск. сапф. и","Серьги, иск. сапф. и","Серьги, иск. изумр. и","Серьги, иск. сапф. и","Серьги, иск. сапф. и","Серьги, иск. сапф. и","Серьги с фианит., ро","Кольцо, иск. изумр. и","Серьги с фианит., ро","Серьги с фиан., родир"
456,Кольцо обр. гл. 22.0,Кольцо обруч. гл. 21.,"Обруч.кольцо, 23 (ш4)",Кольцо обруч. гладкое,Кольцо обруч. гл. 22.,"Кольцо обручальное, 2","Обручальное кольцо, г","Обручальное кольцо, г",Кольцо обруч. гладкое,Кольцо обручальное с,"Обруч.кольцо, 22 (ш5)",Кольцо обручальное гл,Кольцо обручальное с,"Кольцо обручальное, 1",Кольцо обруч. гладкое,"Кольцо обручальное, 1",Кольцо обручальное 22,Кольцо обруч. гладкое,"Обручальное кольцо, г","Кольцо обручальное, 1","Кольцо обручальное, 1"
457,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги
458,Нонна пустотелая БРАС,БРАСЛЕТ ИТАЛЬЯНКА Д 2,БРАСЛЕТ РОЗА 20; ПУСТ,БРАСЛЕТ ИТАЛЬЯНКА 20;,БРАСЛЕТ ИТАЛЬЯНКА 20;,БРАСЛЕТ БИСМАРК Т 23;,БРАСЛЕТ БИСМАРК 23; П,БРАСЛЕТ БИСМАРК 19; П,БРАСЛЕТ БИСМАРК 21; П,БРАСЛЕТ РОМБ 21; ПУСТ,БРАСЛЕТ РОМБ 23; ПУСТ,БРАСЛЕТ РОМБ 18; ПОЛН,БРАСЛЕТ БИСМАРК 22; П,БРАСЛЕТ БИСМАРК 20; П,"БРАСЛЕТ РОМБ 21,0; ПО",БРАСЛЕТ ФАРАОН 19; ПО,БРАСЛЕТ ПАНЦИРЬ 22; П,БРАСЛЕТ РОМБ 17; ПОЛН,БРАСЛЕТ НОННА 19; ПОЛ,БРАСЛЕТ ФАРАОН 20; ПО,БРАСЛЕТ БИСМАРК 18; П
459,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Серьги,Кольцо,Колье,Серьги,Браслет,Браслет,Браслет,Браслет,Браслет,Серьги


In [130]:
recomend[recomend['Material'].str.contains('БРАСЛЕТ')]

,Material,Rec_1,Rec_2,Rec_3,Rec_4,Rec_5,Rec_6,Rec_7,Rec_8,Rec_9,Rec_10,Rec_11,Rec_12,Rec_13,Rec_14,Rec_15,Rec_16,Rec_17,Rec_18,Rec_19,Rec_20
